In [3]:
import sys
sys.path.append("../../reverie/backend_server/")
from dotenv import load_dotenv
load_dotenv('../../reverie/backend_server/.env')

True

In [4]:
from persona.cognitive_modules.retrieve_db import *


## 链接数据库

In [9]:
vector_db = VectorDB(COLLECTION_NAME, EMBEDDING_DIM, "../../reverie/data/test_milvus.db")
vector_db.create_collection(replace=False)


输出 id 为 1 的记录

In [10]:
vector_db.get(ids=[1])

data: ["{'id': 1, 'node_count': 592, 'type_count': 477, 'type': 'event', 'depth': 0, 'expiration': None, 'description': 'Klaus Mueller is sleeping', 'keywords': ['sleep', 'Klaus Mueller'], 'filling': [], 'ori_id': 592, 'created': '2023-02-14 00:02:10', 'embedding_key': 'Klaus Mueller is sleeping', 'object': 'sleep', 'owner': 'Klaus Mueller', 'poignancy': 1, 'predicate': 'is', 'subject': 'Klaus Mueller', 'vector': [0.0052637444, -0.0028237281, 0.011367064, -0.016148709, -0.020189164, 0.021002503, -0.035970557, -0.007634889, -0.0074512316, -0.0071232724, 0.03429141, 0.016043762, 0.013525035, -0.0148893455, -0.013839875, -0.02672211, 0.026354797, -0.005729446, 0.020399058, -0.02954256, 0.008421991, -0.0001444045, 0.0068609053, 0.010179852, 0.007503705, -0.0038371219, -0.0003972405, -0.023271982, -0.029332666, -0.012462447, 0.013682456, -0.016896455, -0.025213499, -0.0013979259, -0.011622872, -0.017040757, 0.0053818095, -0.015925696, 0.015991287, 0.0015758437, 0.0070248847, -0.0006669869, 

## 关键字检索

### Adapter

In [11]:
persona_name = "Isabella Rodriguez"

""" 1. check `retrieve_adapter` """
s = "the Ville:Isabella Rodriguez's apartment:main room:bed"
p = 'used'
o = 'scheduled'

df = retrieve_adapter(vector_db, persona_name, s, p, o)
retrieved = {
    "events": [record for _, record in df.query("type == 'event' ").iterrows()],
    "thoughts": [record for _, record in df.query("type == 'thought' ").iterrows()]
}
perceived = [retrieved["events"][0]]
perceived


[id                                                             594
 node_count                                                     911
 type_count                                                     762
 type                                                         event
 depth                                                            0
 expiration                                                    None
 description                                      bed is being used
 keywords                                               [used, bed]
 filling                                                         []
 ori_id                                                         911
 created                                        2023-02-13 21:11:20
 embedding_key                                    bed is being used
 object                                                        used
 owner                                           Isabella Rodriguez
 poignancy                                      

### retrieve

In [12]:
retrieved = retrieve(persona_name, perceived, vector_db)
retrieved.keys()


dict_keys(['bed is being used'])

In [14]:
key = list(retrieved.keys())[0]
retrieved[key]['curr_envent']


id                                                             594
node_count                                                     911
type_count                                                     762
type                                                         event
depth                                                            0
expiration                                                    None
description                                      bed is being used
keywords                                               [used, bed]
filling                                                         []
ori_id                                                         911
created                                        2023-02-13 21:11:20
embedding_key                                    bed is being used
object                                                        used
owner                                           Isabella Rodriguez
poignancy                                                     

In [15]:
pd.DataFrame(retrieved[key]['events'])

,id,node_count,type_count,type,depth,expiration,description,keywords,filling,ori_id,created,embedding_key,object,owner,poignancy,predicate,subject,vector
0,594,911,762,event,0,None,bed is being used,"[used, bed]",[],911,2023-02-13 21:11:20,bed is being used,used,Isabella Rodriguez,1,be,the Ville:Isabella Rodriguez's apartment:main ...,"[0.016329842, 0.0044551194, 0.0025867892, -0.0..."
1,595,910,761,event,0,None,bed is idle,"[idle, bed]",[],910,2023-02-13 21:11:10,bed is idle,idle,Isabella Rodriguez,1,is,the Ville:Isabella Rodriguez's apartment:main ...,"[-0.0057399357, -0.0039616018, 0.0025072147, -..."
2,601,904,755,event,0,None,desk is being used,"[used, desk]",[],904,2023-02-13 21:02:20,desk is being used,used,Isabella Rodriguez,1,be,the Ville:Isabella Rodriguez's apartment:main ...,"[-0.0051832516, -0.0043148356, 0.004246725, -0..."
3,603,902,753,event,0,None,bed is being used,"[used, bed]",[],902,2023-02-13 20:52:50,bed is being used,used,Isabella Rodriguez,1,be,the Ville:Isabella Rodriguez's apartment:main ...,"[0.016329842, 0.0044551194, 0.0025867892, -0.0..."
4,605,900,751,event,0,None,bed is idle,"[idle, bed]",[],900,2023-02-13 20:47:50,bed is idle,idle,Isabella Rodriguez,1,is,the Ville:Isabella Rodriguez's apartment:main ...,"[-0.0057399357, -0.0039616018, 0.0025072147, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,1473,32,26,event,0,None,bed is idle,"[idle, bed]",[],32,2023-02-13 06:20:40,bed is idle,idle,Isabella Rodriguez,1,is,the Ville:Isabella Rodriguez's apartment:main ...,"[-0.0057399357, -0.0039616018, 0.0025072147, -..."
165,1488,17,11,event,0,None,bed is idle,"[idle, bed]",[],17,2023-02-13 06:00:10,bed is idle,idle,Isabella Rodriguez,1,is,the Ville:Isabella Rodriguez's apartment:main ...,"[-0.0057399357, -0.0039616018, 0.0025072147, -..."
166,1489,16,10,event,0,None,bed is empty,"[empty, bed]",[],16,2023-02-13 06:00:10,bed is empty,empty,Isabella Rodriguez,1,is,the Ville:Isabella Rodriguez's apartment:main ...,"[0.017170861, 0.0071785194, 0.0037103195, -0.0..."
168,1496,9,8,event,0,None,bed is being used,"[used, bed]",[],9,2023-02-13 00:00:10,bed is being used,used,Isabella Rodriguez,1,be,the Ville:Isabella Rodriguez's apartment:main ...,"[0.016329842, 0.0044551194, 0.0025867892, -0.0..."


In [16]:
pd.DataFrame(retrieved[key]['thoughts'])

,id,node_count,type_count,type,depth,expiration,description,keywords,filling,ori_id,created,embedding_key,object,owner,poignancy,predicate,subject,vector
10,620,885,139,thought,2,2023-03-15 20:17:10,Isabella Rodriguez is excited about the Valent...,"[excited, Isabella Rodriguez, be]",[node_10],885,2023-02-13 20:17:10,Isabella Rodriguez is excited about the Valent...,excited,Isabella Rodriguez,9,be,Isabella Rodriguez,"[-0.019704042, -0.014711998, -0.003958637, -0...."
12,623,882,136,thought,9,2023-03-15 20:17:10,Isabella Rodriguez is heavily invested in the ...,"[heavily invested, Isabella Rodriguez, be]","[node_495, node_773, node_682, node_573, node_...",882,2023-02-13 20:17:10,Isabella Rodriguez is heavily invested in the ...,heavily invested,Isabella Rodriguez,8,be,Isabella Rodriguez,"[-0.022664877, -0.023121938, -0.0033909937, -0..."
11,621,884,138,thought,6,2023-03-15 20:17:10,Isabella Rodriguez is a regular at Hobbs Cafe,"[regular, Isabella Rodriguez, be]","[node_687, node_106, node_388, node_114]",884,2023-02-13 20:17:10,Isabella Rodriguez is a regular at Hobbs Cafe,regular,Isabella Rodriguez,6,be,Isabella Rodriguez,"[-0.011845188, -0.010041044, 0.0031859705, -0...."
9,617,888,142,thought,7,2023-03-15 20:17:10,Isabella Rodriguez is heavily involved in the ...,"[be, Isabella Rodriguez, heavily involved]","[node_199, node_495, node_574]",888,2023-02-13 20:17:10,Isabella Rodriguez is heavily involved in the ...,heavily involved,Isabella Rodriguez,8,be,Isabella Rodriguez,"[-0.023854695, -0.018783225, -0.003325637, -0...."
8,615,890,144,thought,7,2023-03-15 20:17:10,Isabella Rodriguez is a hard-working employee ...,"[hard-working employee at Hobbs Cafe, Isabella...","[node_492, node_102]",890,2023-02-13 20:17:10,Isabella Rodriguez is a hard-working employee ...,hard-working employee at Hobbs Cafe,Isabella Rodriguez,6,be,Isabella Rodriguez,"[-0.022065401, -0.010115556, -0.005840048, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,1404,101,7,thought,2,2023-03-15 07:50:10,Isabella Rodriguez is very invested in the Val...,"[Isabella Rodriguez, be, invested]","[node_10, node_7, node_99]",101,2023-02-13 07:50:10,Isabella Rodriguez is very invested in the Val...,invested,Isabella Rodriguez,7,be,Isabella Rodriguez,"[-0.022663765, -0.020398712, -0.004725481, -0...."
149,1403,102,8,thought,2,2023-03-15 07:50:10,Isabella Rodriguez is a hard-working employee ...,"[hard-working employee at Hobbs Cafe, Isabella...","[node_14, node_7, node_72]",102,2023-02-13 07:50:10,Isabella Rodriguez is a hard-working employee ...,hard-working employee at Hobbs Cafe,Isabella Rodriguez,7,be,Isabella Rodriguez,"[-0.022065401, -0.010115556, -0.005840048, -0...."
148,1400,105,11,thought,2,2023-03-15 07:50:10,Isabella Rodriguez is a frequent visitor to Ho...,"[frequent visitor, Isabella Rodriguez, be]","[node_11, node_52, node_61]",105,2023-02-13 07:50:10,Isabella Rodriguez is a frequent visitor to Ho...,frequent visitor,Isabella Rodriguez,6,be,Isabella Rodriguez,"[-0.014230818, -0.009960227, 0.0016308947, -0...."
147,1391,114,20,thought,2,2023-03-15 07:50:10,Isabella Rodriguez is a regular at Hobbs Cafe,"[regular, Isabella Rodriguez, be]","[node_14, node_12]",114,2023-02-13 07:50:10,Isabella Rodriguez is a regular at Hobbs Cafe,regular,Isabella Rodriguez,7,be,Isabella Rodriguez,"[-0.011845188, -0.010041044, 0.0031859705, -0...."


## 向量检索

### Adapter

In [17]:
""" 2.1 emb 检索 """
query_vector = get_embedding("Valentine's Day")
res = retrieve_nodes_adapter(vector_db, [query_vector, np.array(query_vector)*0.8], persona_name)


2024-09-06 02:26:30.704 | DEBUG    | persona.cognitive_modules.retrieve_db:retrieve_nodes_adapter:146 -  owner == "Isabella Rodriguez" and type in ["event", "thought"]


In [18]:
res[0]

,id,distance,poignancy,created,subject,predicate,object,embedding_key,score
18,620,0.837048,9,2023-02-13 20:17:10,Isabella Rodriguez,be,excited,Isabella Rodriguez is excited about the Valent...,4.811144
47,1010,0.829703,9,2023-02-13 14:16:40,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in plan...,4.635684
39,623,0.832173,8,2023-02-13 20:17:10,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,4.596519
63,617,0.827092,8,2023-02-13 20:17:10,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in the ...,4.581276
38,732,0.832173,8,2023-02-13 17:55:10,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,4.535534
45,722,0.830282,8,2023-02-13 17:55:10,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,4.529861
27,823,0.834731,8,2023-02-13 17:10:20,Isabella Rodriguez,be,invested,Isabella Rodriguez is very invested in the Val...,4.524104
62,724,0.827092,8,2023-02-13 17:55:10,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in the ...,4.520291
53,1306,0.827092,9,2023-02-13 09:43:20,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in the ...,4.514565
37,922,0.832173,8,2023-02-13 15:36:00,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,4.476463


In [19]:
res[1]

,id,distance,poignancy,created,subject,predicate,object,embedding_key,score
18,620,0.669638,9,2023-02-13 20:17:10,Isabella Rodriguez,be,excited,Isabella Rodriguez is excited about the Valent...,4.308915
47,1010,0.663762,9,2023-02-13 14:16:40,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in plan...,4.137862
39,623,0.665739,8,2023-02-13 20:17:10,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,4.097216
63,617,0.661674,8,2023-02-13 20:17:10,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in the ...,4.085021
38,732,0.665739,8,2023-02-13 17:55:10,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,4.036230
45,722,0.664226,8,2023-02-13 17:55:10,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,4.031692
62,724,0.661674,8,2023-02-13 17:55:10,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in the ...,4.024035
27,823,0.667785,8,2023-02-13 17:10:20,Isabella Rodriguez,be,invested,Isabella Rodriguez is very invested in the Val...,4.023264
53,1306,0.661674,9,2023-02-13 09:43:20,Isabella Rodriguez,be,heavily involved,Isabella Rodriguez is heavily involved in the ...,4.018310
37,922,0.665739,8,2023-02-13 15:36:00,Isabella Rodriguez,be,heavily invested,Isabella Rodriguez is heavily invested in the ...,3.977159


### new_retrieve

In [20]:
""" 2.2 new_retrieve 测试"""
focal_points = ["Valentine's Day", "sleeping"]
retrieved = new_retrieve(persona_name, focal_points, vector_db, n_count=8)
retrieved.keys()


2024-09-06 02:26:47.481 | DEBUG    | persona.cognitive_modules.retrieve_db:retrieve_nodes_adapter:146 -  owner == "Isabella Rodriguez" and type in ["event", "thought"]


dict_keys(["Valentine's Day", 'sleeping'])

In [21]:
pd.DataFrame(retrieved[focal_points[0]])

,id,distance,poignancy,created,subject,predicate,object,embedding_key,score
18,620,0.837048,9,2023-02-13 20:17:10,Isabella Rodriguez,be,excited,Isabella Rodriguez is excited about the Valent...,4.811144
0,918,0.885115,7,2023-02-13 15:36:00,Isabella Rodriguez,is,scheduled,The Valentine's Day party is scheduled for Feb...,4.363478
16,1116,0.839082,8,2023-02-13 12:28:30,Isabella Rodriguez,prepare,Valentine's Day party,Isabella Rodriguez is preparing for a Valentin...,4.299909
21,811,0.836171,7,2023-02-13 17:10:20,Isabella Rodriguez,set up,cafe,Isabella Rodriguez is setting up the cafe for ...,4.280515
1,1078,0.880702,7,2023-02-13 12:58:10,Isabella Rodriguez,prepare,Valentine's Day party,preparing for the Valentine's Day party,4.244491
13,622,0.842780,6,2023-02-13 20:17:10,Isabella Rodriguez,be involved,planning and preparation of the Valentine's Da...,Isabella Rodriguez is involved in the planning...,4.228339
20,928,0.836171,7,2023-02-13 15:36:00,Isabella Rodriguez,set up,cafe,Isabella Rodriguez is setting up the cafe for ...,4.216644
15,1300,0.839082,8,2023-02-13 09:43:20,Isabella Rodriguez,prepare,Valentine's Day party,Isabella Rodriguez is preparing for a Valentin...,4.191002


In [22]:
pd.DataFrame(retrieved[focal_points[1]])

,id,distance,poignancy,created,subject,predicate,object,embedding_key,score
5,1087,0.858757,5,2023-02-13 12:50:30,the Ville:Isabella Rodriguez's apartment:main ...,be,used,bed is being used,3.872913
7,639,0.858757,3,2023-02-13 19:27:50,the Ville:Isabella Rodriguez's apartment:main ...,be,used,bed is being used,3.633546
8,612,0.858757,2,2023-02-13 20:28:00,the Ville:Isabella Rodriguez's apartment:main ...,be,used,bed is being used,3.458338
0,604,0.875295,1,2023-02-13 20:52:10,Isabella Rodriguez,get,ready,getting ready for bed,3.317945
2,597,0.860817,1,2023-02-13 21:11:10,Isabella Rodriguez,is,sleep,Isabella Rodriguez is sleeping,3.282382
10,594,0.858757,1,2023-02-13 21:11:20,the Ville:Isabella Rodriguez's apartment:main ...,be,used,bed is being used,3.276271
9,603,0.858757,1,2023-02-13 20:52:50,the Ville:Isabella Rodriguez's apartment:main ...,be,used,bed is being used,3.268607
6,657,0.858757,1,2023-02-13 18:51:20,the Ville:Isabella Rodriguez's apartment:main ...,be,used,bed is being used,3.218566
